<a href="https://colab.research.google.com/github/vinagoros/dv_streamprocessing/blob/helio_branch/PS_Hospital_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install river

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pprint import pprint
from river import datasets
from river import compose
from river import linear_model
from river import metrics
from river import evaluate
from river import preprocessing
from river import optim
from river import feature_extraction
from river import stats
from river import ensemble
from river import neighbors, stream, tree
import numpy as np
import sklearn
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
import matplotlib.pyplot as plt
import datetime as dt

In [ ]:
mypath = '/content/drive/MyDrive/Teaching/Streams processing/Data/Emergency department/Portugal/sns_df.csv'
df = pd.read_csv(mypath, sep=',',parse_dates=['Acquisition_Time'])
df.head()

,Unnamed: 0,Acquisition_Time,Hospital,Urgency_Type,Service,Emergency_Stage,Waiting_Time,People_Waiting,H_Name
0,0,2017-11-15 00:01:10,211,Urgncia Polivalente,Espera: Medicina Interna,4,17,0,S Jose
1,1,2017-11-15 00:01:10,211,Urgncia Polivalente,Espera: Medicina Interna,3,34,0,S Jose
2,2,2017-11-15 00:01:10,211,Urgncia Polivalente,Espera: Cirurgia Geral,3,6,0,S Jose
3,3,2017-11-15 00:01:10,211,Urgncia Polivalente,Espera: Oftalmologia,3,5,0,S Jose
4,4,2017-11-15 00:01:10,211,Urgncia Polivalente,Espera: Medicina Interna,2,309,8,S Jose


In [ ]:
df2=df.loc[df['H_Name']=='Santa Maria']
df2.reset_index(drop=True, inplace=True)

In [ ]:
Lag_value = 3 ## a data point every 10 mn -> we look at the value on the last 30 mn (3*10 mn)
df3=df2[['Service','Emergency_Stage','Acquisition_Time','People_Waiting','Waiting_Time']].copy()
df3["lag_Waiting_Time_30_mn"] = 0
for em in range(1,6):
  df3.loc[(df3['Service']=="Cirurgia") & (df3['Emergency_Stage']==em),'lag_Waiting_Time_30_mn'] = df3[(df3['Service']=="Cirurgia") & (df3['Emergency_Stage']==em) ].Waiting_Time.shift(Lag_value)
  df3.loc[(df3['Service']=="Medicina") & (df3['Emergency_Stage']==em),'lag_Waiting_Time_30_mn'] = df3[(df3['Service']=="Medicina") & (df3['Emergency_Stage']==em) ].Waiting_Time.shift(Lag_value)
#new_df2['lag(Waiting_Time,30 mn)'] = new_df2[new_df2['Service']=="Medicina"].Waiting_Time.shift(3)
df3[['Service','Emergency_Stage','Waiting_Time','lag_Waiting_Time_30_mn']].head(25)

,Service,Emergency_Stage,Waiting_Time,lag_Waiting_Time_30_mn
0,Cirurgia,4,16,NaN
1,Medicina,4,12,NaN
2,Cirurgia,3,26,NaN
3,Medicina,3,33,NaN
4,Cirurgia,2,30,NaN
5,Medicina,2,144,NaN
6,Medicina,1,270,NaN
7,Cirurgia,4,16,NaN
8,Medicina,4,12,NaN
9,Cirurgia,3,26,NaN


In [ ]:
df4 = df3.dropna().copy()
df4.sample(10)

,Service,Emergency_Stage,Acquisition_Time,People_Waiting,Waiting_Time,lag_Waiting_Time_30_mn
134693,Cirurgia,3,2018-05-01 14:27:37,2,14,20.0
146832,Medicina,4,2018-05-17 01:34:04,0,17,36.0
56850,Medicina,4,2018-01-24 23:47:44,2,28,17.0
191985,Cirurgia,1,2018-07-12 19:01:22,2,49,207.0
182776,Cirurgia,4,2018-07-01 16:48:46,1,80,80.0
1906,Cirurgia,1,2017-11-17 22:43:01,0,564,263.0
190224,Medicina,4,2018-07-10 16:25:24,0,25,10.0
302579,Cirurgia,1,2018-12-16 17:06:41,0,9,9.0
305235,Medicina,1,2018-12-20 01:32:50,2,358,236.0
13319,Medicina,3,2017-12-01 06:50:13,1,27,12.0


In [ ]:
df4[df4['lag_Waiting_Time_30_mn'].isna()], df4['Service'].unique()

(Empty DataFrame
 Columns: [Service, Emergency_Stage, Acquisition_Time, People_Waiting, Waiting_Time, lag_Waiting_Time_30_mn]
 Index: [], array(['Cirurgia', 'Medicina', 'Aguarda Balcao (Cirurgia)',
        'Aguarda Balcao (Medico)', 'Aguarda Balcao Trauma',
        'Aguarda Sector Ortopedico', 'Aguarda Atendimento Geral',
        'Espera por 1 observacao medica', 'Espera: Pediatria',
        'Espera: Cirurgia', 'Espera: Otorrino',
        'Aguarda Pequena Cirurgia Geral'], dtype=object))

In [ ]:
services_to_keep = ['Cirurgia', 'Medicina']
df5 = df4.query("Service in @services_to_keep")
df5[df5['Service']=='Aguarda Balcao Trauma']

,Service,Emergency_Stage,Acquisition_Time,People_Waiting,Waiting_Time,lag_Waiting_Time_30_mn


In [ ]:
df6 = df5.copy()
y = df6.pop('Waiting_Time')


In [ ]:
### Simple linear model with only two engineered features

def get_hour_and_weekday(x):
    x['weekday'] = float(x['Acquisition_Time'].weekday()) # Mon:0,Tue:1,...,Sat:6,Sun:7
    x['hour'] = x['Acquisition_Time'].hour
    return x


model = (compose.Select('Service') | preprocessing.OneHotEncoder())
model += compose.Select('Emergency_Stage','lag_Waiting_Time_30_mn')
model += (get_hour_and_weekday |
    feature_extraction.TargetAgg(by='hour', how=stats.Mean()))

model |= preprocessing.StandardScaler()
model |= linear_model.LinearRegression(optimizer=optim.SGD(0.001))

metric = metrics.MAE()
model

Pipeline (
  TransformerUnion (
    Pipeline (
      Select (
        Service
      ),
      OneHotEncoder (
        sparse=False
      )
    ),
    Select (
      Emergency_Stage
      lag_Waiting_Time_30_mn
    ),
    Pipeline (
      FuncTransformer (
        func="get_hour_and_weekday"
      ),
      TargetAgg (
        by=['hour']
        how=Mean ()
        target_name="y"
      )
    )
  ),
  StandardScaler (
    with_std=True
  ),
  LinearRegression (
    optimizer=SGD (
      lr=Constant (
        learning_rate=0.001
      )
    )
    loss=Squared ()
    l2=0.
    intercept_init=0.
    intercept_lr=Constant (
      learning_rate=0.01
    )
    clip_gradient=1e+12
    initializer=Zeros ()
  )
)

In [ ]:
evaluate.progressive_val_score(
    dataset= stream.iter_pandas(df6,y),
    model=model,
    metric=metric,
    moment='Acquisition_Time',
    delay=dt.timedelta(minutes=30),
    print_every=20_000
)


[20,000] MAE: 20.039801
[40,000] MAE: 21.228047
[60,000] MAE: 22.328992
[80,000] MAE: 23.948694
[100,000] MAE: 24.142692
[120,000] MAE: 24.307408
[140,000] MAE: 24.840487
[160,000] MAE: 25.1471
[180,000] MAE: 24.346577
[200,000] MAE: 23.99823
[220,000] MAE: 24.262698
[240,000] MAE: 24.535265
[260,000] MAE: 24.970172
[280,000] MAE: 25.000706
[300,000] MAE: 25.209572
[320,000] MAE: 25.136054
[340,000] MAE: 25.628401
[360,000] MAE: 26.010871
[380,000] MAE: 26.311844


MAE: 26.744229